# Modulos

In [ ]:
import os
from pathlib import Path
import pandas as pd

# Rutas

In [ ]:
#R_Pila_I_SIE = r"C:\Users\crist\Downloads\reporte_2025_11_11 09_56.txt"
R_Pila_I_SIE = r"C:\Users\osmarrincon\Downloads\reporte_2025_12_02 09_53.txt"

fecha = "02-12-2025"

#R_Salida = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\Pila_SIE\Pila I\{fecha}.txt"
R_Salida = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\Pila_SIE\Pila I\{fecha}.txt"

# Proceso depuración
## Columnas

In [ ]:
# Leer el archivo .txt como texto plano
with open(R_Pila_I_SIE, encoding='ansi') as f:
    lines = f.readlines()

# Separar encabezados y registros
header = lines[0].strip().split('|')
data_lines = lines[1:]

# Validaciones
expected_cols = 91
error_rows = []
corrected_lines = []

for idx, line in enumerate(data_lines):
    cols = line.strip().split('|')
    
    # Si la línea tiene más columnas de las esperadas
    if len(cols) > expected_cols:
        
        # Encuentra el índice de la columna que causa el error. 
        # Aquí, se asume que es el valor 'null' en la posición 30.
        try:
            null_index = cols.index('null', 30, 32) # Buscamos 'null' entre las posiciones 30 y 32
            
            # Si se encuentra, lo eliminamos de la lista
            del cols[null_index]
            
            # Ahora la lista de columnas tiene el tamaño correcto.
            corrected_lines.append('|'.join(cols))
            
            # Guardar el registro original con error para el reporte
            error_rows.append((cols[0], cols[2], cols[3])) # ["N° Radicación o PILA", "N° Identificación Cotizante", "Tipo Cotizante"]
            
        except ValueError:
            # Si no se encuentra un 'null' en ese rango, no es el error que buscamos.
            # En este caso, simplemente recortamos la línea a 91 columnas como antes
            corrected_lines.append('|'.join(cols[:expected_cols]))
            print(f"Alerta: Registro en línea {idx+2} tiene más de {expected_cols} columnas, pero no se encontró 'null' para corregir. Se recortó a {expected_cols} columnas.")

    # Si la línea tiene el número de columnas esperado, la agregamos tal cual
    elif len(cols) == expected_cols:
        corrected_lines.append('|'.join(cols))
    
    # Si la línea tiene menos columnas, también la agregamos (esto podría ser un error, 
    # pero el código original no lo trata como tal, solo imprime una alerta).
    else:
        print(f"Alerta: Registro en línea {idx+2} tiene {len(cols)} columnas. Se recortó a {expected_cols} columnas.")
        corrected_lines.append('|'.join(cols[:expected_cols]))


# Imprimir registros con error
# ... (El resto del código para imprimir y guardar el archivo es el mismo)
if error_rows:
    print("Registros corregidos por columna 'Departamento' con pipe extra:")
    for r in error_rows:
        print(f"N° Radicación o PILA: {r[0]}, N° Identificación Cotizante: {r[1]}, Tipo Cotizante: {r[2]}")
    print(f"Total registros corregidos: {len(error_rows)}")
else:
    print("No se encontraron registros con error de columna 'Departamento'.")

# Guardar el archivo corregido
with open(R_Salida, 'w', encoding='ansi') as f_out:
    f_out.write('|'.join(header) + '\n')
    for line in corrected_lines:
        f_out.write(line + '\n')

print(f"Archivo corregido guardado en: {R_Salida}")
print(f"Total de registros procesados: {len(corrected_lines)}")